In [1]:
import numpy as np
import pandas as pd
import gc
import time

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

import warnings
from time import time
import xgboost as xgb
xgb.__version__

'1.3.3'

In [2]:
%%time
train = pd.read_csv('../input/train_1830.csv')

CPU times: user 43.8 s, sys: 4.1 s, total: 47.9 s
Wall time: 47.9 s


In [3]:
application_train = pd.read_csv('../input/application_train.csv')
target = application_train.TARGET.values

In [4]:
good_features = train.columns[2:]

In [11]:
params =  {'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 'lambda': 0.07382375583625363,
 'alpha': 0.0023919654661293415,
 'colsample_bytree': 0.8157006290650177,
 'subsample': 0.8206021006500338,
 'learning_rate': 0.001,
 'max_depth': 9,
 'min_child_weight': 269,
 'eval_metric': 'auc'}



In [12]:
%%time
start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].values[train_index], train[good_features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=100000, evals=watchlist, 
                verbose_eval=100, 
                early_stopping_rounds = 1500)
        
        #preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        #y_oof[test_index] = ypred
        #preds_total += preds
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        
        break

1
[0]	train-auc:0.76740	eval-auc:0.74165
[100]	train-auc:0.79053	eval-auc:0.76453
[200]	train-auc:0.79168	eval-auc:0.76528
[300]	train-auc:0.79239	eval-auc:0.76571
[400]	train-auc:0.79292	eval-auc:0.76599
[500]	train-auc:0.79336	eval-auc:0.76631
[600]	train-auc:0.79380	eval-auc:0.76662
[700]	train-auc:0.79424	eval-auc:0.76692
[800]	train-auc:0.79467	eval-auc:0.76725
[900]	train-auc:0.79507	eval-auc:0.76760
[1000]	train-auc:0.79542	eval-auc:0.76793
[1100]	train-auc:0.79568	eval-auc:0.76821
[1200]	train-auc:0.79600	eval-auc:0.76855
[1300]	train-auc:0.79628	eval-auc:0.76885
[1400]	train-auc:0.79651	eval-auc:0.76913
[1500]	train-auc:0.79670	eval-auc:0.76938
[1600]	train-auc:0.79688	eval-auc:0.76961
[1700]	train-auc:0.79707	eval-auc:0.76984
[1800]	train-auc:0.79719	eval-auc:0.77002
[1900]	train-auc:0.79734	eval-auc:0.77022
[2000]	train-auc:0.79750	eval-auc:0.77041
[2100]	train-auc:0.79770	eval-auc:0.77067
[2200]	train-auc:0.79789	eval-auc:0.77091
[2300]	train-auc:0.79812	eval-auc:0.77114
[2

In [8]:
%%time
params['eta'] = 0.01


start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].values[train_index], train[good_features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=10000, evals=watchlist, 
                verbose_eval=100, 
                early_stopping_rounds = 200)
        
        #preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        #y_oof[test_index] = ypred
        #preds_total += preds
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        
        break

1
[0]	train-auc:0.75684	eval-auc:0.74114
[100]	train-auc:0.78491	eval-auc:0.76299
[200]	train-auc:0.79218	eval-auc:0.76740
[300]	train-auc:0.80006	eval-auc:0.77176
[400]	train-auc:0.80895	eval-auc:0.77676
[500]	train-auc:0.81755	eval-auc:0.78116
[600]	train-auc:0.82484	eval-auc:0.78477
[700]	train-auc:0.83085	eval-auc:0.78738
[800]	train-auc:0.83591	eval-auc:0.78906
[900]	train-auc:0.84048	eval-auc:0.79052
[1000]	train-auc:0.84436	eval-auc:0.79169
[1100]	train-auc:0.84800	eval-auc:0.79270
[1200]	train-auc:0.85152	eval-auc:0.79344
[1300]	train-auc:0.85490	eval-auc:0.79407
[1400]	train-auc:0.85808	eval-auc:0.79463
[1500]	train-auc:0.86112	eval-auc:0.79512
[1600]	train-auc:0.86395	eval-auc:0.79556
[1700]	train-auc:0.86694	eval-auc:0.79590
[1800]	train-auc:0.86973	eval-auc:0.79619
[1900]	train-auc:0.87237	eval-auc:0.79647
[2000]	train-auc:0.87486	eval-auc:0.79671
[2100]	train-auc:0.87728	eval-auc:0.79698
[2200]	train-auc:0.87959	eval-auc:0.79720
[2300]	train-auc:0.88180	eval-auc:0.79735
[2

In [9]:
%%time
params['eta'] = 0.03


start_time = time()
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].values[train_index], train[good_features].values[test_index]
        y_train, y_val = target[train_index], target[test_index]
        xgtrain = xgb.DMatrix(xgtrain, label = y_train ) 
        xgval = xgb.DMatrix(xgval, label = y_val ) 
        watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
        clf = xgb.train(params, xgtrain, num_boost_round=10000, evals=watchlist, 
                verbose_eval=100, 
                early_stopping_rounds = 100)
        
        #preds = clf.predict(xgtest)
        ypred = clf.predict(xgval, ntree_limit=clf.best_ntree_limit)
        #y_oof[test_index] = ypred
        #preds_total += preds
        #np.save("preds_hist_"+str(i), preds)
        #p.save("oof_preds_hist_"+str(i), ypred)
        print('Time elapsed:', time()-start_time)
        #clf.save_model('xgb_gpu_hist_1812_2_fold_'+str(i)+'.model')
        del clf
        gc.collect()
        i +=1
        print('Fold AUC:', roc_auc_score(y_val, ypred))
        
        break

1
[0]	train-auc:0.75684	eval-auc:0.74114
[100]	train-auc:0.79963	eval-auc:0.77163
[200]	train-auc:0.82433	eval-auc:0.78354
[300]	train-auc:0.83969	eval-auc:0.78941
[400]	train-auc:0.85126	eval-auc:0.79219
[500]	train-auc:0.86071	eval-auc:0.79379
[600]	train-auc:0.86889	eval-auc:0.79458
[700]	train-auc:0.87669	eval-auc:0.79524
[800]	train-auc:0.88416	eval-auc:0.79571
[900]	train-auc:0.89045	eval-auc:0.79580
[979]	train-auc:0.89508	eval-auc:0.79579
Time elapsed: 60.63681197166443
Fold AUC: 0.7958909365808279
CPU times: user 1min 6s, sys: 3.36 s, total: 1min 9s
Wall time: 1min


In [18]:
train[good_features].loc[[1,5,99]]

,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,...,total_credit_None,debt_credit_ratio_None,AGE_INT,ORGANIZATION_TYPE_cnt,region_id,prev_app_decision_days_max,prev_app_days_approved_decision_max,neighbors_target_mean_500,neighbors_500_credit_downpayment_median,neighbors_500_DAYS_CREDIT_mean_median
1,0.0,0.0,0.0,0.0,0.0,0.0959,0.0968,0.0924,0.0529,0.0529,...,1017400.5,0.000000,45,10180,0.003541,-746.0,-746.0,0.040080,-165012.75,NaN
5,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,468445.5,0.512454,46,19390,0.035792,-82.0,-370.0,0.032064,-49896.00,NaN
99,NaN,NaN,NaN,NaN,NaN,0.1361,0.1374,0.0777,0.0761,0.0761,...,NaN,NaN,61,64648,0.026392,-1105.0,-1105.0,0.046092,-66825.00,NaN


In [19]:
kf = StratifiedKFold(5, shuffle=True, random_state=1974)
i=1
for train_index, test_index in kf.split(train,target):
    #if i < 4:
        print(i)
        xgtrain, xgval = train[good_features].loc[train_index], train[good_features].loc[test_index]
        y_train, y_val = target[train_index], target[test_index]
        
        break

1


In [20]:
xgtrain.head()

,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,...,total_credit_None,debt_credit_ratio_None,AGE_INT,ORGANIZATION_TYPE_cnt,region_id,prev_app_decision_days_max,prev_app_days_approved_decision_max,neighbors_target_mean_500,neighbors_500_credit_downpayment_median,neighbors_500_DAYS_CREDIT_mean_median
0,0.0,0.0,0.0,0.0,1.0,0.0247,0.025,0.0252,0.0369,0.0369,...,865055.565,0.284122,25,78832,0.018801,-606.0,-606.0,0.278557,-58236.75,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,52,78832,0.008019,-181.0,-181.0,0.034068,-18000.00,NaN
4,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,146250.000,0.000000,54,97,0.028663,-374.0,-374.0,0.072144,-78588.00,NaN
5,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,468445.500,0.512454,46,19390,0.035792,-82.0,-370.0,0.032064,-49896.00,NaN
6,0.0,0.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,...,4800811.500,0.224410,37,78832,0.035792,-74.0,-74.0,0.010020,-163469.25,NaN


In [22]:
xgtrain['target'] = y_train
xgval['target'] = y_val

In [23]:
xgtrain.to_csv('../input/xgtrain.csv', index=False)
xgval.to_csv('../input/xgval.csv', index=False)